In [ ]:
import argparse
import os
from collections import defaultdict

import pandas as pd
import torch
from sklearn.metrics.pairwise import cosine_similarity
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

from configs import DATA_DIR, DEVICE
from main import main
from noise_data_filter import noise_labeling
from utils import set_seed

/data/ephemeral/eyeol/.local/share/virtualenvs/data-centric-0i6SbDDw/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
def get_sentence_label(model: AutoModelForSequenceClassification, tokenizer: AutoTokenizer, sentences: list[str]):
    # Tokenize sentences
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(DEVICE)

    # Classify sentence labels
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Predict label
    if len(sentences) == 1:
        predicted_class = logits.argmax(dim=1).item()  # 한 문장일 경우 단일 값 반환
    else:
        predicted_class = logits.argmax(dim=1).tolist()

    return predicted_class

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # 텍스트를 토큰화하고, PyTorch 텐서 형식으로 반환
        inputs = self.tokenizer(
            text, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt"
        )

        # Trainer가 요구하는 형식으로 딕셔너리 생성
        item = {key: val.squeeze(0) for key, val in inputs.items()}  # input_ids와 attention_mask
        item["labels"] = torch.tensor(label, dtype=torch.long)  # 레이블 추가
        return item

In [ ]:
def train_classifier(
    model,
    tokenizer,
    train_texts,
    train_labels,
    output_dir="./results",
    epochs=3,
    batch_size=8,
    learning_rate=2e-5,
):
    # 편의상 val_dataset은 그냥 train_dataset과 동일하게 세팅
    # 어차피 학습 후에 re-label 능력을 보는게 중요
    train_dataset = CustomDataset(train_texts, train_labels, tokenizer)
    val_dataset = CustomDataset(train_texts, train_labels, tokenizer)

    # TrainingArguments 설정
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        learning_rate=learning_rate,
        evaluation_strategy="epoch" if val_dataset else "no",
        save_strategy="epoch",
        metric_for_best_model="accuracy",
    )

    # 평가 지표 정의 (예: 정확도)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = logits.argmax(axis=-1)
        accuracy = (predictions == labels).mean()
        return {"accuracy": accuracy}

    # Trainer 설정
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )

    # 학습 시작
    trainer.train()

    # 학습이 완료된 모델 반환
    return model

In [ ]:
restored_with_filtered = pd.read_csv("data/restored_with_filtered.csv")

In [ ]:
restored_with_filtered.head()

,ID,text,target,restored,sim_score,noise_ratio,noise_label
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,정의 : 파이 미사일 KT 이용기간 2년 단위 분쟁 종결,0.540519,0.450000,True
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,K찰.국제도시 개발 Lwo 로 L3한 N% 회장 2 T0&}송사 개시,0.777942,0.450000,True
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,김정은 자주통일 새정치 나가야1보,0.676605,0.357143,True
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,1.000000,0.000000,False
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,미대선 앞두고 R2F드andan 발…비해 감시 강화,0.690566,0.352941,True


In [ ]:
noise_data = restored_with_filtered[
    (restored_with_filtered["noise_label"])
    & (0.3 <= restored_with_filtered["noise_ratio"])
    & (restored_with_filtered["noise_ratio"] <= 0.5)
]

In [ ]:
# 1084개
train_texts = noise_data["restored"].tolist()
train_labels = noise_data["target"].tolist()

In [ ]:
model_name = "jhgan/ko-sroberta-multitask"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=7).to(DEVICE)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jhgan/ko-sroberta-multitask and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 모델 학습
model = train_classifier(model, tokenizer, train_texts, train_labels)

not_noise = restored_with_filtered[restored_with_filtered["noise_label"] == False]

to_classify = not_noise["restored"].tolist()

/data/ephemeral/eyeol/.local/share/virtualenvs/data-centric-0i6SbDDw/lib/python3.10/site-packages/transformers/training_args.py:1559: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_351663/3899532758.py:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.983786,0.739852
2,No log,0.613338,0.847786
3,No log,0.487382,0.887454


In [ ]:
classified_labels = get_sentence_label(model, tokenizer, to_classify)

In [ ]:
not_noise["target"] = classified_labels

/tmp/ipykernel_351663/1156828708.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_noise["target"] = classified_labels


In [ ]:
not_noise.head()

,ID,text,target,restored,sim_score,noise_ratio,noise_label
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,1.0,0.000000,False
5,ynat-v1_train_00005,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,6,美성인 6명 중 1명꼴 배우자·연인 빚 떠안은 적 있다,1.0,0.000000,False
7,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,1,아가메즈 33득점 우리카드 KB손해보험 완파…3위 굳...,1.0,0.133333,False
8,ynat-v1_train_00008,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,2,朴대통령 얼마나 많이 놀라셨어요…경주 지진현장 방문종합,1.0,0.000000,False
9,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,4,듀얼심 아이폰 하반기 출시설 솔솔…알뜰폰 기대감,1.0,0.000000,False


In [ ]:
restored_with_filtered.head()

,ID,text,target,restored,sim_score,noise_ratio,noise_label
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,정의 : 파이 미사일 KT 이용기간 2년 단위 분쟁 종결,0.540519,0.450000,True
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,K찰.국제도시 개발 Lwo 로 L3한 N% 회장 2 T0&}송사 개시,0.777942,0.450000,True
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,김정은 자주통일 새정치 나가야1보,0.676605,0.357143,True
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,1.000000,0.000000,False
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,미대선 앞두고 R2F드andan 발…비해 감시 강화,0.690566,0.352941,True


In [ ]:
noise_data = restored_with_filtered[restored_with_filtered["noise_label"] == True]
restored_with_filtered.loc[noise_data.index, "text"] = noise_data["restored"]

In [ ]:
restored_with_filtered.head()

,ID,text,target,restored,sim_score,noise_ratio,noise_label
0,ynat-v1_train_00000,정의 : 파이 미사일 KT 이용기간 2년 단위 분쟁 종결,4,정의 : 파이 미사일 KT 이용기간 2년 단위 분쟁 종결,0.540519,0.450000,True
1,ynat-v1_train_00001,K찰.국제도시 개발 Lwo 로 L3한 N% 회장 2 T0&}송사 개시,3,K찰.국제도시 개발 Lwo 로 L3한 N% 회장 2 T0&}송사 개시,0.777942,0.450000,True
2,ynat-v1_train_00002,김정은 자주통일 새정치 나가야1보,2,김정은 자주통일 새정치 나가야1보,0.676605,0.357143,True
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,1.000000,0.000000,False
4,ynat-v1_train_00004,미대선 앞두고 R2F드andan 발…비해 감시 강화,6,미대선 앞두고 R2F드andan 발…비해 감시 강화,0.690566,0.352941,True


In [ ]:
not_noise_target = not_noise[["ID", "target"]]

In [ ]:
# df_final과 not_noise_target을 ID를 기준으로 병합하여 target 값 업데이트
restored_with_filtered = restored_with_filtered.merge(not_noise_target, on="ID", how="left", suffixes=("", "_new"))

In [ ]:
# 덮어쓰기를 위해 df_final의 target 값을 새로 병합된 target 값으로 업데이트
restored_with_filtered["target"] = (
    restored_with_filtered["target_new"].combine_first(restored_with_filtered["target"]).astype(int)
)

In [ ]:
restored_with_filtered

,ID,text,target,restored,sim_score,noise_ratio,noise_label,target_new
0,ynat-v1_train_00000,정의 : 파이 미사일 KT 이용기간 2년 단위 분쟁 종결,4,정의 : 파이 미사일 KT 이용기간 2년 단위 분쟁 종결,0.540519,0.450000,True,NaN
1,ynat-v1_train_00001,K찰.국제도시 개발 Lwo 로 L3한 N% 회장 2 T0&}송사 개시,3,K찰.국제도시 개발 Lwo 로 L3한 N% 회장 2 T0&}송사 개시,0.777942,0.450000,True,NaN
2,ynat-v1_train_00002,김정은 자주통일 새정치 나가야1보,2,김정은 자주통일 새정치 나가야1보,0.676605,0.357143,True,NaN
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,1.000000,0.000000,False,4.0
4,ynat-v1_train_00004,미대선 앞두고 R2F드andan 발…비해 감시 강화,6,미대선 앞두고 R2F드andan 발…비해 감시 강화,0.690566,0.352941,True,NaN
...,...,...,...,...,...,...,...,...
2794,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,1.000000,0.000000,False,6.0
2795,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,4,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,1.000000,0.000000,False,4.0
2796,ynat-v1_train_02797,텔레그램 한 달 등급을 한 번-down…C버스 정기버스 39종! 2보,4,텔레그램 한 달 등급을 한 번-down…C버스 정기버스 39종! 2보,0.654411,0.347826,True,NaN
2797,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1.000000,0.000000,False,1.0


In [ ]:
restored_with_filtered = restored_with_filtered[["ID", "text", "target"]]

In [ ]:
restored_with_filtered.head()

,ID,text,target
0,ynat-v1_train_00000,정의 : 파이 미사일 KT 이용기간 2년 단위 분쟁 종결,4
1,ynat-v1_train_00001,K찰.국제도시 개발 Lwo 로 L3한 N% 회장 2 T0&}송사 개시,3
2,ynat-v1_train_00002,김정은 자주통일 새정치 나가야1보,2
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,4
4,ynat-v1_train_00004,미대선 앞두고 R2F드andan 발…비해 감시 강화,6


In [ ]:
main(restored_with_filtered)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss


Training Summary
------------------------------
Train Data Count     : 1959
Eval Data Count      : 840
Train Loss           : 1.2871
Eval Loss            : 0.9197
Eval F1              : 0.7487


Predicting: 100%|██████████| 30000/30000 [03:51<00:00, 129.33it/s]
